# Fox, Goose and Bag of Beans Puzzle

[From Wikipedia](https://en.wikipedia.org/wiki/Fox,_goose_and_bag_of_beans_puzzle):

"Once upon a time a farmer went to a market and purchased a fox, a goose, and a bag of beans. On his way home, the farmer came to the bank of a river and rented a boat. But crossing the river by boat, the farmer could carry only himself and a single one of his purchases: the fox, the goose, or the bag of beans.

"If left unattended together, the fox would eat the goose, or the goose would eat the beans.

"The farmer's challenge was to carry himself and his purchases to the far bank of the river, leaving each purchase intact. How did he do it?"



In [105]:
from pyomo.environ import *
from pyomo.gdp import *

T = 7

model = ConcreteModel()

model.I = Set(initialize=['fox', 'goose', 'beans'])

model.a = Var(range(0,T+1), model.I, domain=Binary)
model.b = Var(range(0,T+1), model.I, domain=Binary)
model.x = Var(range(1,T+1), model.I, domain=Binary)

model.obj = Objective(expr=sum(model.b[T, i] for i in model.I), sense=maximize)

model.ic1 = Constraint(model.I, rule=lambda model, i: model.a[0,i] == 1)
model.ic2 = Constraint(model.I, rule=lambda model, i: model.b[0,i] == 0)
model.boat = Constraint(range(1,T+1), rule=lambda model, t: sum(model.x[t,i] for i in model.I) <= 1)

model.cons = ConstraintList()
for i in model.I:
    model.cons.add(model.a[1,i] == model.a[0,i] - model.x[1,i])
    model.cons.add(model.b[1,i] == model.b[0,i] + model.x[1,i])
    
    for t in range(2,T+1,2):
        model.cons.add(model.b[t,i] == model.b[t-1,i] - model.x[t,i])
        model.cons.add(model.a[t,i] == model.a[t-1,i] + model.x[t,i])
        model.cons.add(model.a[t+1,i] == model.a[t,i] - model.x[t+1,i])
        model.cons.add(model.b[t+1,i] == model.b[t,i] + model.x[t+1,i])
        
model.gfa = Constraint(range(1,T), rule=lambda model, t: 
                       model.a[t,'goose'] + model.a[t,'fox'] <= 1)
model.gfb = Constraint(range(1,T), rule=lambda model, t: 
                       model.b[t,'goose'] + model.b[t,'fox'] <= 1)
model.gba = Constraint(range(1,T), rule=lambda model, t: 
                       model.a[t,'goose'] + model.a[t,'beans'] <= 1)
model.gbb = Constraint(range(1,T), rule=lambda model, t: 
                       model.b[t,'goose'] + model.b[t,'beans'] <= 1)


SolverFactory('glpk').solve(model)

print("\nStarting Condition:")
print("\tShoreline A:", [i for i in model.I if model.a[0,i]()])
print("\tShoreline B:", [i for i in model.I if model.b[0,i]()])

destination = [' from B to A', ' from A to B']

for t in range(1,T+1):
    print("\nTrip " + str(t) + destination[t%2] + ":", [i for i in model.I if model.x[t,i]()])
    print("\tShoreline A:", [i for i in model.I if model.a[t,i]()])
    print("\tShoreline B:", [i for i in model.I if model.b[t,i]()])



Starting Condition:
	Shoreline A: ['beans', 'goose', 'fox']
	Shoreline B: []

Trip 1 from A to B: ['goose']
	Shoreline A: ['beans', 'fox']
	Shoreline B: ['goose']

Trip 2 from B to A: []
	Shoreline A: ['beans', 'fox']
	Shoreline B: ['goose']

Trip 3 from A to B: []
	Shoreline A: ['beans', 'fox']
	Shoreline B: ['goose']

Trip 4 from B to A: []
	Shoreline A: ['beans', 'fox']
	Shoreline B: ['goose']

Trip 5 from A to B: []
	Shoreline A: ['beans', 'fox']
	Shoreline B: ['goose']

Trip 6 from B to A: []
	Shoreline A: ['beans', 'fox']
	Shoreline B: ['goose']

Trip 7 from A to B: ['beans']
	Shoreline A: ['fox']
	Shoreline B: ['beans', 'goose']


In [38]:
model.ic1 = Constraint(model.GOODS, rule=lambda m,j: m.a[0,j] == 1)
model.ic2 = Constraint(model.GOODS, rule=lambda m,j: m.b[0,j] == 0)

model.dyn = ConstraintList()
for t in range(0,tf):
    for k in model.GOODS:
        model.dyn.add(model.a[t+1,k] == model.a[t,k] - model.x[t+1,k])
        model.dyn.add(model.b[t+1,k] == model.b[t,k] + model.x[t+1,k])




for t in model.T:
    print("\nTrip = ", t)
    print("Shoreline A:", [k for k in model.GOODS if model.a[t,k]()])
    print("Shoreline B:", [k for k in model.GOODS if model.b[t,k]()])
    print("Boat Takes:", [k for k in model.GOODS if model.x[t+1,k]()])

model.pprint()


Trip =  0
Shoreline A: ['beans', 'goose', 'fox']
Shoreline B: []
Boat Takes: ['beans']

Trip =  1
Shoreline A: ['goose', 'fox']
Shoreline B: ['beans']
Boat Takes: []

Trip =  2
Shoreline A: ['goose', 'fox']
Shoreline B: ['beans']
Boat Takes: ['goose', 'fox']

Trip =  3
Shoreline A: []
Shoreline B: ['beans', 'goose', 'fox']
Boat Takes: []

Trip =  4
Shoreline A: []
Shoreline B: ['beans', 'goose', 'fox']
Boat Takes: []

Trip =  5
Shoreline A: []
Shoreline B: ['beans', 'goose', 'fox']


KeyError: "Index '(6, 'beans')' is not valid for indexed component 'x'"